# NFL Big Data Bowl 2024

## Importing dataview libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
DATA_ROOT = '../data/'

In [4]:
games = pd.read_csv(DATA_ROOT + 'games.csv')
players = pd.read_csv(DATA_ROOT + 'players.csv')
plays = pd.read_csv(DATA_ROOT + 'plays.csv')
tackles = pd.read_csv(DATA_ROOT + 'tackles.csv')

In [67]:
# Modify the plays dataframe to add a column for the play direction (pass short middle, run left end, etc.) and a column for if the play was in shotgun formation.
import re

def playDescriptionToDirection(play_description):
    play = re.search('(pass (?:short|deep) (?:left|middle|right))|((?:left|right) (?:guard|tackle|end))|(up the middle)', play_description)
    return play.group(0) if play else 'FUMBLED BALL'

def playDescriptionToShotgun(play_description):
    return re.search('[(]Shotgun[)]', play_description) != None

def playDescriptionToStartingYard(play_description, offense):
    if re.search('TOUCHDOWN', play_description):
        return 100 - int(re.search('(?<=for )\d+', play_description).group(0))
    if re.search('SAFETY', play_description):
        return 100 + int(re.search('(?<=for -)\d+', play_description).group(0))
    match = re.search('(?:(?:to|at)\s)(?:(?:(\w+)(?:\s)(\d+))|(50))(?: for )(?:(?:([-]?\d+)(?: yard))|(no gain))', play_description)
    if match == None:
        print('WARN: Couldn\'t parse:', play_description)
        return -1
    end_yard = -1
    if match.group(3) == '50':
        end_yard = 50
    else:
        end_yard = int(match.group(2))
        end_yard = 100 - end_yard if match.group(1) != offense else end_yard
    if match.group(5) == 'no gain':
        return end_yard
    return end_yard - int(match.group(4))
    

In [68]:
print('Direction')
plays['playDirection'] = plays['playDescription'].map(playDescriptionToDirection)
print('Shotgun')
plays['shotgun'] = plays['playDescription'].map(playDescriptionToShotgun)
print('Starting Yardage')
plays['startingYard'] = list(map(playDescriptionToStartingYard, plays['playDescription'], plays['possessionTeam']))

Direction
Shotgun
Starting Yardage


In [63]:
print(plays[['playDescription', 'playDirection', 'shotgun', 'startingYard']].head(5))

                                     playDescription      playDirection  \
0  (7:52) (Shotgun) M.Mariota pass short middle t...  pass short middle   
1  (7:38) (Shotgun) C.Claypool right end to PIT 3...          right end   
2  (8:57) D.Carr pass short middle to D.Waller to...  pass short middle   
3  (13:12) M.Boone left tackle to DEN 44 for 7 ya...        left tackle   
4  (8:33) D.Singletary right guard to TEN 32 for ...        right guard   

   shotgun  startingYard  
0     True            41  
1     True            34  
2    False            30  
3    False            37  
4    False            65  
